In [1]:
import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model


class CPLEXCCPPSolver:
    def __init__(self, params):
        self.solution = None
        self.params = params
        
    def solve(self):
        mdl = Model('CCPP', checker=False)
        (K, V, V_count, theta, L, l_i, V_loc_x, V_loc_y, E, d, TV, SV, G) = self.params
        # binary dict of possibilities of placing controller theta in place of a node v
        tv = mdl.binary_var_dict(TV, name="tv")

        #binary dict if switch connected to controller in place v
        sv = mdl.binary_var_dict(SV, name="sv")
        # minimize max const of edge for given cluster
        #mdl.minimize(mdl.max(d[i,j]*sv[i,j] for i,j in SV))

        # or minimize average const
        mdl.minimize((mdl.sum(d[i,j]*sv[i,j] for i,j in SV))/V_count)
        
        # exactly k controllers
        mdl.add_constraints(mdl.sum(tv[t,v] for v in V)==1 for t in theta)

        # max one controller in one place
        mdl.add_constraints(mdl.sum(tv[t,v] for t in theta)<=1 for v in V)

        # one switch has only one link (link to controller)
        mdl.add_constraint(mdl.sum(sv[i,j] for i, j in SV) == V_count )

        mdl.add_constraints(mdl.sum(sv[i,j] for j in V) == 1 for i in V)

        # if switch has active link, it must have controller at it's end
        mdl.add_indicator_constraints(mdl.indicator_constraint(sv[i,j],mdl.sum(tv[t,j] for t in theta)==1) for i,j in SV)

        # maximum capacity of a switch cannot be exceeded 
        mdl.add_indicator_constraints(
            mdl.indicator_constraint(tv[t,v],mdl.sum(sv[i,j]*l_i[i] for i,j in SV if j==v)<=L[t]) for v in V for t in theta)
        mdl.parameters.timelimit = 30
        solution = mdl.solve()
        if solution:
            self.solution = solution
        else:
            print("No solution for given constraints")
        
    
        
    def plotSolution(self):
        if self.solution:    
            colors = {}
            fig, ax = plt.subplots(figsize=(15,15))
            (K, V, V_count, theta, L, l_i, V_loc_x, V_loc_y, E, d, TV, SV, G) = self.params
        
            active_arcs = [a for a in SV if sv[a].solution_value>0.9]

            plt.scatter(V_loc_x[:], V_loc_y[:])
            for i in V:
                plt.annotate('   $%s$' % (list(G.nodes)[i]), (V_loc_x[i], V_loc_y[i]))
            for i,j in active_arcs:
                path = nx.shortest_path(G,list(G.nodes)[i], list(G.nodes)[j], 'weight')
                if not path[-1] in colors:
                    colors[path[-1]] = np.random.rand(3,)

                for k in range(0, len(path)-1):
                    vi = list(G.nodes).index(path[k])
                    vj = list(G.nodes).index(path[k+1])
                    plt.plot([V_loc_x[vi], V_loc_x[vj]], [V_loc_y[vi], V_loc_y[vj]],c = colors[path[-1]],lw=5)
                    if k == len(path)-2:
                        plt.plot(V_loc_x[vj], V_loc_y[vj], c=colors[path[-1]], marker='s', markersize=15)
            plt.axis('equal')

In [2]:
# #create summary of each controller
# if solution:    
#     theta_results = {}
#     for t in theta:
#         theta_result = {'place': -1, 'switches': [], 'capacity_used': 0, 'capacity_max': L[t]}
#         for v in V:
#             if tv[t, v].solution_value > 0.9:
#                 theta_result['place'] = v
#                 break

#         for i, j in active_arcs:
#             if j == theta_result['place']:
#                 theta_result['switches'].append(i)
#                 theta_result['capacity_used']+=l_i[i]

#         theta_results[t] = theta_result 
#     theta_results